In [153]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
# from torch.utils.tensorboard import SummaryWriter

In [154]:
df = pd.read_csv('./data/hackathon_train.csv', encoding='cp949', index_col=0)

In [155]:
df

,User_ID,Gender,Age,MBTI,Q_number,Answer
Data_ID,,,,,,
1,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...
2,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...
3,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...
4,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...
5,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...
...,...,...,...,...,...,...
14384,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...
14385,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...
14386,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...


In [156]:
# let's include the "Question" column as well
# read the "Question" column using pandas
df_question = pd.read_excel('./data/Question.xlsx', sheet_name='Sheet1', index_col=0)
df_question['Question']

index
1     주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁...
2     자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하나요? 요즘 어떤 관심...
3     다른 사람이 울고 있는 모습을 보면 자신도 울고 싶어질 때가 많나요? 이런 상황에서...
4            일이 잘못될 때를 대비해 여러 대비책을 세우는 편인가요? 이유는 무엇인가요.
5          압박감이 심한 환경에서도 평정심을 유지하는 편인가요? 최근 경험을 말씀해주세요.
6     파티나 행사에서 새로운 사람에게 먼저 자신을 소개하기보다는 주로 이미 알고 있는 사...
7     하나의 프로젝트를 완전히 완료한 후 다른 프로젝트를 시작하는 편인가요? 경험에 비추...
8                      매우 감상적인 편인가요? 그렇게 생각한 이유는 무엇인가요.
9     일정이나 목록으로 계획을 세우는 일을 좋아하나요? 예를 들어 다이어리나 전자적 도구...
10    작은 실수로도 자신의 능력이나 지식을 의심하곤 하나요? 최근에 그러한 경험이 있다면...
11    관심이 가는 사람에게 다가가서 대화를 시작하기가 어렵지 않고 쉽나요? 대부분 어떠했...
12    예술 작품의 다양한 해석에 대해 토론하는 일에는 크게 관심이 없나요? 이유는 무엇인가요.
13    감정보다는 이성을 따르는 편인가요? 본인이 판단했을 때 생각이 궁금하네요. 주변의 ...
14    하루 일정을 계획하기보다는 즉흥적으로 하고 싶은 일을 하는 것을 좋아하나요? 경험을...
15    다른 사람에게 자신이 어떤 사람으로 보일지 걱정하지 않는 편인가요? 그렇게 생각한 ...
16             단체 활동에 참여하는 일을 즐기나요? 요즘 어떤지 자세히 말씀해주세요. 
17    결말을 자신의 방식으로 해석할 수 있는 책과 영화를 좋아하나요? 답변의 이유때문에 ...
18    자신보다는 남의 성공을 돕는 일에서 더 큰 만족감을 느끼나요? 답

In [157]:
df['Question'] = ''

In [158]:
# include the "Question" column in the original dataframe
# df['Question'] = df_question['Question']
# using the "Q_number" column as the index, let's paste the "Question" column to the dataframe
for i, row in df.iterrows():
    df.loc[i, 'Question'] = df_question.loc[row['Q_number'], 'Question']

In [159]:
df

,User_ID,Gender,Age,MBTI,Q_number,Answer,Question
Data_ID,,,,,,,
1,1,1,30,INFP,1,<아니다> 어릴 때 왕따 당한 경험이 있고 외부 활동을 좋아하지 않기 때문에 소수의...,주기적으로 새로운 친구를 만드나요? 경험을 비추어봤을 때 어떤지와 그러한 이유가 궁...
2,1,1,30,INFP,2,<중립> 다양한 관심사를 탐구하진 않지만 대체로 자연과 역사에 관련된 것을 좋아하...,자유 시간 중 상당 부분을 다양한 관심사를 탐구하는 데 할애하나요? 요즘 어떤 관심...
3,1,1,30,INFP,3,<그렇다> 감정 이입이 잘되어 코미디 영화에서 사람이 울고 있을 때도 울기 때문에 ...,다른 사람이 울고 있는 모습을 보면 자신도 울고 싶어질 때가 많나요? 이런 상황에서...
4,1,1,30,INFP,4,<중립> 대비책을 세우긴 하는데 세우다가 마는 편입니다. 일의 변수가 생길 수 있고...,일이 잘못될 때를 대비해 여러 대비책을 세우는 편인가요? 이유는 무엇인가요.
5,1,1,30,INFP,5,<아니다> 평정심을 유지 못 하는 편입니다. 머릿속은 백지화가 된 상태로 말도 제대...,압박감이 심한 환경에서도 평정심을 유지하는 편인가요? 최근 경험을 말씀해주세요.
...,...,...,...,...,...,...,...
14384,240,0,40,ISTJ,44,<그렇다> 저는 계획에 차질이 생기면 돌아가기 위해 노력을 합니다. 이유는 그 계획...,계획에 차질이 생기면 최대한 빨리 계획으로 돌아가기 위해 노력하나요? 최근에 있었던...
14385,240,0,40,ISTJ,45,<그렇다> 저는 예전의 실수를 후회할 때가 많습니다. 이유는 그만큼 나태하게 산 적...,오래전의 실수를 후회할 때가 많나요? 요즘에 무엇때문에 그런가요.
14386,240,0,40,ISTJ,46,<아니다> 저는 인간의 존재와 삶의 이유에 대해 깊이 생각하지 않습니다. 이유는 이...,인간의 존재와 삶의 이유에 대해 깊이 생각하지 않는 편인가요? 답변에 대한 이유를 ...


In [160]:
# load pretrained model
def get_model():
    # * Model          | Tokenizer          | Pretrained weights shortcut
    # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
    model = BertModel.from_pretrained("kykim/bert-kor-base")
    n_hl = model.config.num_hidden_layers
    embed_dim = model.config.embedding_size
    return model, tokenizer, n_hl, embed_dim

In [161]:
model, tokenizer, n_hl, embed_dim = get_model()

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [162]:
# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:40])
    test_df_list.append(df[df['User_ID']==idx][40:])

In [163]:
train_df = pd.concat(train_df_list, ignore_index=True)
test_df = pd.concat(test_df_list, ignore_index=True)

In [164]:
# tokenize questions
train_question_tokens = tokenizer(train_df['Question'].tolist(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
test_question_tokens = tokenizer(test_df['Question'].tolist(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)

# tokenize answers
train_answer_tokens = tokenizer(train_df['Answer'].tolist(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
test_answer_tokens = tokenizer(test_df['Answer'].tolist(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)

# merge question and answer tokens
combined_train_tokens = {}
for key in train_question_tokens.keys():
    combined_train_tokens[key] = torch.cat((train_question_tokens[key], train_answer_tokens[key]), dim=1)
combined_test_tokens = {}
for key in test_question_tokens.keys():
    combined_test_tokens[key] = torch.cat((test_question_tokens[key], test_answer_tokens[key]), dim=1)



In [165]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [166]:
def build_dataloader(tokens, batch_size=32, shuffle=False):
    ds = MyMapDataset(tokens)
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle)

train_question_dataloader = build_dataloader(train_question_tokens, batch_size=32, shuffle=False)
test_question_dataloader = build_dataloader(test_question_tokens, batch_size=32, shuffle=False)
train_answer_dataloader = build_dataloader(train_answer_tokens, batch_size=32, shuffle=False)
test_answer_dataloader = build_dataloader(test_answer_tokens, batch_size=32, shuffle=False)

In [167]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [168]:
train_result_question = forward(model, train_question_dataloader, device=0)
test_result_question = forward(model, test_question_dataloader, device=1)
train_result_answer = forward(model, train_answer_dataloader, device=0)
test_result_answer = forward(model, test_answer_dataloader, device=1)

In [171]:
train_result_question[0].shape, train_result_question[1].shape, train_result_answer[0].shape, train_result_answer[1].shape

(torch.Size([9600, 768]),
 torch.Size([9600, 768]),
 torch.Size([9600, 768]),
 torch.Size([9600, 768]))

In [172]:
test_result_question[0].shape, test_result_question[1].shape, test_result_answer[0].shape, test_result_answer[1].shape

(torch.Size([1920, 768]),
 torch.Size([1920, 768]),
 torch.Size([1920, 768]),
 torch.Size([1920, 768]))

In [170]:
# save the results
if not os.path.exists('./embedded'):
    os.mkdir('./embedded')
torch.save(train_result_question, './embedded/train_result_question.pt')
torch.save(test_result_question, './embedded/test_result_question.pt')
torch.save(train_result_answer, './embedded/train_result_answer.pt')
torch.save(test_result_answer, './embedded/test_result_answer.pt')